In [1]:
import QuantLib as ql
evDate = ql.Date(11, 7, 2011)
settings = ql.Settings.instance()
settings.setEvaluationDate(evDate)
DC360 = ql.Actual360()
DC30360 = ql.Thirty360()
Cal = ql.NullCalendar()
BDC = ql.Unadjusted
DGR = ql.DateGeneration.Backward
startDate = ql.Date(13, 7, 2012)
maturityDate = ql.Date(13, 7, 2013)
begDate = maturityDate
endDate = ql.Date(13, 7, 2018)
refDate = startDate
period = ql.Period(ql.Semiannual)
sch = ql.Schedule(begDate, endDate, period, Cal, BDC, BDC, DGR, False)
USD = ql.USDCurrency()
settlementDays = 2
fixingDays = 2
r = 0.05
faceAmt = 100.0
flatTS = ql.FlatForward(0, Cal, r, DC360, ql.Compounded, ql.Annual)
h_flatTS = ql.YieldTermStructureHandle(flatTS)
Idx6M = ql.IborIndex("Index", period, fixingDays, USD, Cal, BDC, False, DC360, h_flatTS)

In [2]:
A = 0.1
Sigma = 0.01
HW = ql.HullWhite(h_flatTS, A, Sigma)

In [3]:
P1Y = ql.Period(1, ql.Years)
P2Y = ql.Period(2, ql.Years)
P3Y = ql.Period(3, ql.Years)
P4Y = ql.Period(4, ql.Years)
P5Y = ql.Period(5, ql.Years)
h_Q1X5 = ql.QuoteHandle(ql.SimpleQuote(0.11480))
h_Q2X4 = ql.QuoteHandle(ql.SimpleQuote(0.11080))
h_Q3X3 = ql.QuoteHandle(ql.SimpleQuote(0.10700))
h_Q4X2 = ql.QuoteHandle(ql.SimpleQuote(0.10210))
h_Q5X1 = ql.QuoteHandle(ql.SimpleQuote(0.10000))

In [4]:
SHelper1X5 = ql.SwaptionHelper(P1Y, P5Y, h_Q1X5, Idx6M, P1Y, DC30360, DC360, h_flatTS)
SHelper2X4 = ql.SwaptionHelper(P2Y, P4Y, h_Q2X4, Idx6M, P1Y, DC30360, DC360, h_flatTS)
SHelper3X3 = ql.SwaptionHelper(P3Y, P3Y, h_Q3X3, Idx6M, P1Y, DC30360, DC360, h_flatTS)
SHelper4X2 = ql.SwaptionHelper(P4Y, P2Y, h_Q4X2, Idx6M, P1Y, DC30360, DC360, h_flatTS)
SHelper5X1 = ql.SwaptionHelper(P5Y, P1Y, h_Q5X1, Idx6M, P1Y, DC30360, DC360, h_flatTS)

In [7]:
JamshiEngine = ql.JamshidianSwaptionEngine(HW, h_flatTS)
SHelper1X5.setPricingEngine(JamshiEngine)
SHelper2X4.setPricingEngine(JamshiEngine)
SHelper3X3.setPricingEngine(JamshiEngine)
SHelper4X2.setPricingEngine(JamshiEngine)
SHelper5X1.setPricingEngine(JamshiEngine)

In [8]:
NoCon = ql.NoConstraint()
MaxInter = 120
MaxStat = 100
RootEps = 0.01
FuncEps = 0.01
FuncGradEps = 0.01
EndCrit = ql.EndCriteria(MaxInter, MaxStat, RootEps, FuncEps, FuncGradEps)

In [9]:
EpsFunc = 0.000001
Xtol = 0.00000001
Gtol = 0.00000001
LMOpt = ql.LevenbergMarquardt(EpsFunc, Xtol, Gtol)

In [11]:
Vec_Weight = ql.DoubleVector([0.2, 0.2, 0.2, 0.2, 0.2])
Vec_FixParam = ql.BoolVector([False, False])
Vec_SHelper = ql.CalibrationHelperVector([SHelper1X5, SHelper2X4, SHelper3X3, SHelper4X2, SHelper5X1])

In [12]:
h_HW = ql.ShortRateModelHandle(HW)
h_HW.calibrate(Vec_SHelper, LMOpt, EndCrit, NoCon, Vec_Weight, Vec_FixParam)

In [14]:
Params = h_HW.params()

In [15]:
P_A = Params[0]
P_Sigma = Params[1]
print(P_A)
print(P_Sigma)

0.04030544672494135
0.005656779268187906


In [16]:
print(len(Params))

2
